In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [16]:
df=pd.read_csv('/content/Geographicaldata.csv')

In [17]:
df.head()

,City Description,Feature Extraction
0,Abohar is a city located in the Fazilka distri...,"agricultural significance, annual agricultural..."
1,Addanki is a town in the Prakasam district of ...,"Historical temples, Handloom weaving industry."
2,Adilabad is a city located in the Adilabad dis...,"Natural beauty, Kala Ashram cultural center pr..."
3,Adoni is a city in the Kurnool district of And...,"Historical and cultural heritage, important co..."
4,Agartala is the capital city of the state of T...,"Rich cultural heritage, Historical significance."


In [18]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb (Optional)(Use Weights and Biases API to build better models faster)
! pip install transformers[torch]
! pip install accelerate -U

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: ` pip install wandb (Optional)(Use Weights and Biases API to build better models faster)'


In [19]:
!pip install wandb
# log in to WandB
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
   import wandb
   wandb.login()

wandb: Currently logged in as: dandamudigeeta-priya (dandamudigeeta-priya-christ-university). Use `wandb login --relogin` to force relogin


In [20]:
import torch
import numpy as np
import datasets
from transformers import (
   AutoModelForSeq2SeqLM,
   AutoTokenizer,
   Seq2SeqTrainingArguments,
   Seq2SeqTrainer,
   DataCollatorForSeq2Seq,
)
from tabulate import tabulate
import nltk
from datetime import datetime

In [21]:
#model_name to indicate the specific pretrained model used
pretrained_model_name = "sshleifer/distilbart-xsum-12-3"
# Load the pretrained model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
# Tokenization parameters
encoder_max_len = 256
decoder_max_len = 6

In [10]:
from datasets import Dataset

In [22]:
df=pd.read_csv('/content/Geographicaldata.csv')

In [23]:
# Convert DataFrame to Dataset
geographical_data = Dataset.from_pandas(df)
def flatten_example(example):
   return {
       "CityDescription": example["City Description"],
       "FeatureExtraction": example["Feature Extraction"],
   }
def filter_samples(example):
   CityDescription = []
   FeatureExtraction = []
   for desc, feat in zip(example["City Description"], example["Feature Extraction"]):
       if len(desc) > 0:
           CityDescription.append(desc)
           FeatureExtraction.append(feat)
   return {"CityDescription": CityDescription, "FeatureExtraction": FeatureExtraction}



In [24]:
# Apply transformations to the dataset
geographical_data = geographical_data.map(flatten_example)
geographical_data = geographical_data.map(filter_samples, batched=True)


Map:   0%|          | 0/299 [00:00<?, ? examples/s]

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

In [25]:
# Split the dataset into train and validation sets
train_data_txt, validation_data_txt = geographical_data.train_test_split(test_size=0.1).values()

In [26]:
def preprocess_batch(batch, tokenizer, max_source_length, max_target_length):
   src, tgt = batch["CityDescription"], batch["FeatureExtraction"]
   src_tokenized = tokenizer(
       src, padding="max_length", truncation=True, max_length=max_source_length
   )
   tgt_tokenized = tokenizer(
       tgt, padding="max_length", truncation=True, max_length=max_target_length
   )
   batch = {k: v for k, v in src_tokenized.items()}
   # Ignore padding in the loss
   batch["labels"] = [
       [-100 if token == tokenizer.pad_token_id else token for token in l]
       for l in tgt_tokenized["input_ids"]
   ]
   return batch


In [27]:
train_data = train_data_txt.map(
   lambda batch: preprocess_batch(
       batch, tokenizer, encoder_max_len, decoder_max_len
   ),
   batched=True,
   remove_columns=train_data_txt.column_names,
)


Map:   0%|          | 0/269 [00:00<?, ? examples/s]

In [28]:
validation_data = validation_data_txt.map(
   lambda batch: preprocess_batch(
       batch, tokenizer, encoder_max_len, decoder_max_len
   ),
   batched=True,
   remove_columns=validation_data_txt.column_names,
)


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [29]:
import numpy as np
from datasets import load_metric
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
nltk.download("punkt", quiet=True)
metric = load_metric("rouge") #Recall-Oriented Understudy for Gisting Evaluation
def postprocess_text(preds, labels):
   preds = [pred.strip() for pred in preds]
   labels = [label.strip() for label in labels]
   # rougeLSum expects newline after each sentence
   preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
   labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
   return preds, labels

<ipython-input-29-13c42cffecff>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge") #Recall-Oriented Understudy for Gisting Evaluation


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [32]:
def calculate_metrics(eval_preds):
   preds, labels = eval_preds
   if isinstance(preds, tuple):
       preds = preds[0]
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   # Replace -100 in the labels as we can't decode them.
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
   # Some simple post-processing
   decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
   result = metric.compute(
       predictions=decoded_preds, references=decoded_labels, use_stemmer=True
   )
   # Extract a few results from ROUGE
   result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

   prediction_lens = [
       np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
   ]
   result["gen_len"] = np.mean(prediction_lens)
   result = {k: round(v, 4) for k, v in result.items()}
   return result


In [33]:
training_args = Seq2SeqTrainingArguments(
   output_dir="results",
   num_train_epochs=1,  # demo
   do_train=True,
   do_eval=True,
   per_device_train_batch_size=4,  # demo
   per_device_eval_batch_size=4,
   # learning_rate=3e-05,
   warmup_steps=500,
   weight_decay=0.1,
   label_smoothing_factor=0.1,
   predict_with_generate=True,
   logging_dir="logs",
   logging_steps=50,
   save_total_limit=3,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   data_collator=data_collator,
   train_dataset=train_data,
   eval_dataset=validation_data,
   tokenizer=tokenizer,
   compute_metrics=calculate_metrics,
)


In [34]:
trainer.evaluate()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


{'eval_loss': 6.714873313903809,
 'eval_rouge1': 4.0296,
 'eval_rouge2': 0.0,
 'eval_rougeL': 4.0104,
 'eval_rougeLsum': 4.0137,
 'eval_gen_len': 19.3667,
 'eval_runtime': 192.7836,
 'eval_samples_per_second': 0.156,
 'eval_steps_per_second': 0.041}

In [ ]:
trainer.train()

Step,Training Loss
50,5.723800


Step,Training Loss
50,5.723800


In [ ]:
trainer.evaluate()

In [39]:
def generate_summary(test_data, model):
   inputs = tokenizer(
       test_data["CityDescription"],
       padding="max_length",
       truncation=True,
       max_length=encoder_max_len,
       return_tensors="pt",
   )
   input_ids = inputs.input_ids.to(model.device)
   attention_mask = inputs.attention_mask.to(model.device)
   outputs = model.generate(input_ids, attention_mask=attention_mask)
   output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
   return outputs, output_str
model_before_fine_tuning = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
test_samples = validation_data_txt.select(range(10))
features_before_fine_tuning = generate_summary(test_samples, model_before_fine_tuning)[1]
features_after_fine_tuning = generate_summary(test_samples, model)[1]

In [40]:
print("\nTarget Feature:\n")
print(
   tabulate(list(enumerate(test_samples["FeatureExtraction"])), headers=["Id", "Target Feature"])
)
print("\nCity Description:\n")
print(tabulate(list(enumerate(test_samples["CityDescription"])), headers=["Id", "City Description"]))


Target Feature:

  Id  Target Feature
----  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   0  Historical monuments, cultural festival.
   1  Major industrial and commercial center, Educational institutions and recreational centers.
   2  Historical significance, natural landscapes
   3  Historical and cultural heritage, Nagari Hills and Nagari Narasimha Swamy Temple.
   4  Major industrial and commercial center, Blend of urban amenities and historical sites.
   5  The two features of the city are its historical significance and the presence of educational institutions, healthcare facilities, and markets.
   6  Natural beauty including Sundarbans mangrove forest and Ganges delta, rich cultural heritage reflected in folk music, dance, and festivals, and textile and handloom industries.
   7  Religious significance, ancient history.
   8  Historical signifi